<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=8214a9f27698dcd011e33bb5b44099fb2b5d922a209e94144f71433dfb897b7c
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.1
    Uninstalling protobuf-6.33.1:
      Successfully uninstalled protobuf-6.33.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-12-04 12:43:47
-------------------
qualified stocks: 86
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.28 C
CY Investment:  1.55 C
Reserve:  4.95 L
Current:  1.44 C
-------------------
Today PnL: 15.51 K (0.11%)
Current PnL: -24.36 L (-15.73%)
CY Booked + Current PnL: -10.07 L (-6.5%)
-------------------
Total profit:  1.33 L
Total loss:  -25.69 L
-------------------
Total Booked + Current PnL: 16.91 L (13.23%)
Total Booked PnL: 41.27 L (32.29%)
Curr Year Booked PnL: 14.29 L (9.93%)
Prev Year Booked PnL: 26.98 L (21.11%)
Est FTT:  2.33 C
Est FTT PnL: 88.99 L (61.83%)
Deployed:  1.28 C
Current:  1.44 C
CAGR/XIRR %: 6.76%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDIGOPNTS,1408.00,1.13,-7.65,8.30,0.02,13381.0,-13347.0,161212.0,146.09,68.0,M-SC,15.57,234.0,-1.00,1.16,40.23,OX40N,NTT,PAINTS
77,TTKPRESTIG,770.00,-0.06,-16.75,20.24,0.09,16980.0,-16885.0,83892.0,94.66,37.0,M-SC,2.75,253.0,-0.99,0.60,10.22,OX40N,NTT,DURABLES
43,ITC,452.00,0.36,-1.79,12.45,10.44,24471.0,-3584.0,196554.0,-41.15,43.0,X-LC,1.58,5.0,-0.15,1.41,3.67,X40,NTT,FMCG
35,ICICIGI,2252.93,0.37,5.61,13.81,20.19,25425.0,9783.0,184103.0,-17.20,47.0,X-MC,6.61,68.0,0.38,1.32,21.57,X40,ATH,INSURANCE
50,MASFIN,398.61,-2.92,-4.87,28.29,22.05,26369.0,-4770.0,93210.0,-17.80,50.0,H-SC,6.85,164.0,-0.18,0.67,35.49,XR,ATH,FINANCE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
80,VALIANTORG,838.00,4.99,-44.28,212.57,74.15,193766.0,-72451.0,91154.0,-264.07,43.0,H-SC,19.99,149.0,-0.37,0.66,16.84,XR,NTT,CHEMICALS
29,HAPPSTMNDS,1480.71,3.04,-36.05,189.06,84.85,154954.0,-46205.0,81960.0,-23.94,56.0,H-SC,13.99,147.0,-0.30,0.59,5.19,AR,ATH,IT
13,BSOFT,831.70,2.51,-21.43,91.92,50.79,101575.0,-30149.0,110504.0,0.89,74.0,H-SC,5.56,151.0,-0.30,0.80,28.66,XR,ATH,IT
15,CAMS,4762.00,2.37,3.54,20.52,24.79,53511.0,8917.0,260773.0,-1.25,55.0,X-SC,2.15,86.0,0.17,1.88,27.50,X40N,NTT,MISC
67,SONACOMS,806.63,1.58,-13.61,61.52,39.54,53765.0,-13764.0,87395.0,-32.50,55.0,M-SC,6.26,220.0,-0.26,0.63,23.24,AR,ATH,AUTO


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
24,EASEMYTRIP,26.40,-4.76,-56.90,238.03,45.70,181403.0,-100605.0,76210.0,-20.98,52.0,M-SC,23.54,205.0,-0.55,0.55,9.85,XY24,NTT,TRAVEL
44,JCHAC,2282.00,-3.20,-33.57,50.53,-0.01,42898.0,-42909.0,84896.0,17269.57,8.0,M-SC,0.97,233.0,-1.00,0.61,0.00,OX40N,NTT,AC
50,MASFIN,398.61,-2.92,-4.87,28.29,22.05,26369.0,-4770.0,93210.0,-17.80,50.0,H-SC,6.85,164.0,-0.18,0.67,35.49,XR,ATH,FINANCE
28,GREENPANEL,537.00,-1.96,-36.02,123.72,43.14,144919.0,-65943.0,117135.0,162.27,28.0,M-SC,7.15,240.0,-0.46,0.84,10.18,XY24,NTT,MISC
51,MEDANTA,1486.00,-1.93,-0.72,24.98,24.08,31483.0,-916.0,126034.0,-8.70,37.0,X-SC,5.96,89.0,-0.03,0.91,18.75,XY24,NTT,HEALTHCARE


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
38,INDIAMART,4810.62,-0.59,-0.49,107.73,106.72,132225.0,-599.0,122737.0,-50.82,38.0,H-SC,1.57,139.0,-0.00,0.88,22.78,AR,ATH,MISC
37,IEX,219.00,-0.12,0.92,47.27,48.63,95390.0,1845.0,201799.0,-32.92,66.0,H-SC,14.72,136.0,0.02,1.45,12.58,XR,NTT,MISC


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,0.47,-5.60,120.25,107.91,167594.0,-8273.0,139371.0,-23.39,35.0,M-SC,11.29,216.0,-0.05,1.00,0.47,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,1.13,-7.65,8.30,0.02,13381.0,-13347.0,161212.0,146.09,68.0,M-SC,15.57,234.0,-1.00,1.16,40.23,OX40N,NTT,PAINTS
77,TTKPRESTIG,770.00,-0.06,-16.75,20.24,0.09,16980.0,-16885.0,83892.0,94.66,37.0,M-SC,2.75,253.0,-0.99,0.60,10.22,OX40N,NTT,DURABLES
47,KANSAINER,340.00,-1.39,-23.43,48.19,13.47,99510.0,-63171.0,206496.0,-68.51,32.0,H-SC,2.37,158.0,-0.63,1.49,5.00,XY24,NTT,PAINTS
72,TATAELXSI,9161.00,1.38,-24.39,74.63,32.03,74387.0,-32161.0,99674.0,-18.22,48.0,H-SC,7.86,148.0,-0.43,0.72,11.64,OX40N,NTT,IT


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.0,1.10,5.61,63.11,72.27,112939.0,9508.0,178956.0,-9.26,70.0,M-LC,3.06,99.0,0.08,1.29,12.13,XR,NTT,IT
37,IEX,219.0,-0.12,0.92,47.27,48.63,95390.0,1845.0,201799.0,-32.92,66.0,H-SC,14.72,136.0,0.02,1.45,12.58,XR,NTT,MISC


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,1.10,5.61,63.11,72.27,112939.0,9508.0,178956.0,-9.26,70.0,M-LC,3.06,99.0,0.08,1.29,12.13,XR,NTT,IT
37,IEX,219.00,-0.12,0.92,47.27,48.63,95390.0,1845.0,201799.0,-32.92,66.0,H-SC,14.72,136.0,0.02,1.45,12.58,XR,NTT,MISC
38,INDIAMART,4810.62,-0.59,-0.49,107.73,106.72,132225.0,-599.0,122737.0,-50.82,38.0,H-SC,1.57,139.0,-0.00,0.88,22.78,AR,ATH,MISC
85,ZYDUSLIFE,1286.17,-0.13,-1.53,37.74,35.63,77880.0,-3215.0,206359.0,-15.41,41.0,H-MC,3.42,119.0,-0.04,1.48,14.88,AR,ATH,PHARMA
25,FINCABLES,1641.55,0.47,-5.60,120.25,107.91,167594.0,-8273.0,139371.0,-23.39,35.0,M-SC,11.29,216.0,-0.05,1.00,0.47,OX40N,ATH,CABLES


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
9,BAJAJHFL,181.50,-0.48,-21.07,88.08,48.45,139395.0,-42246.0,158260.0,-26.75,13.0,X-MC,9.41,64.0,-0.30,1.14,0.00,X40N,ATH,FINANCE
11,BATAINDIA,2096.00,-0.63,-37.35,115.80,35.20,93351.0,-48056.0,80614.0,5.70,25.0,X-SC,14.38,92.0,-0.51,0.58,0.68,X40,NTT,FOOTWEAR
22,DIXON,18931.72,-1.33,-11.17,36.87,21.58,45899.0,-15651.0,124488.0,-55.24,26.0,X-MC,5.73,56.0,-0.34,0.90,10.21,X40N,ATH,IT
8,AWL,485.00,-0.34,-21.92,96.60,53.50,227588.0,-66153.0,235598.0,-62.23,27.0,X-MC,1.50,58.0,-0.29,1.70,4.64,XY24,NTT,FMCG
23,DMART,5391.45,0.17,-6.93,37.66,28.12,73746.0,-14587.0,195820.0,-19.69,30.0,X-LC,4.21,19.0,-0.20,1.41,15.05,X40N,ATH,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
3,ACC,3906.0,-1.01,-23.45,114.43,64.15,208446.0,-55791.0,182160.0,-55.44,42.0,X-SC,1.48,82.0,-0.27,1.31,2.18,XY24,BTT,CEMENT
8,AWL,485.0,-0.34,-21.92,96.60,53.50,227588.0,-66153.0,235598.0,-62.23,27.0,X-MC,1.50,58.0,-0.29,1.70,4.64,XY24,NTT,FMCG
43,ITC,452.0,0.36,-1.79,12.45,10.44,24471.0,-3584.0,196554.0,-41.15,43.0,X-LC,1.58,5.0,-0.15,1.41,3.67,X40,NTT,FMCG
15,CAMS,4762.0,2.37,3.54,20.52,24.79,53511.0,8917.0,260773.0,-1.25,55.0,X-SC,2.15,86.0,0.17,1.88,27.50,X40N,NTT,MISC
36,ICICIPRULI,790.0,0.21,1.55,28.97,30.97,51994.0,2739.0,179477.0,-22.29,49.0,X-MC,2.21,75.0,0.05,1.29,14.31,X40,ATH,INSURANCE


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
9,BAJAJHFL,181.50,-0.48,-21.07,88.08,48.45,139395.0,-42246.0,158260.0,-26.75,13.0,X-MC,9.41,64.0,-0.30,1.14,0.00,X40N,ATH,FINANCE
55,QUESS,424.00,-1.06,-30.86,106.60,42.84,47911.0,-20061.0,44945.0,-54.82,35.0,X-SC,37.32,83.0,-0.42,0.32,0.00,XY24,NTT,MISC
19,COLPAL,3726.84,0.56,-19.75,76.33,41.51,161331.0,-52005.0,211360.0,-4.32,34.0,X-MC,7.37,61.0,-0.32,1.52,0.56,XY25,ATH,FMCG
11,BATAINDIA,2096.00,-0.63,-37.35,115.80,35.20,93351.0,-48056.0,80614.0,5.70,25.0,X-SC,14.38,92.0,-0.51,0.58,0.68,X40,NTT,FOOTWEAR
53,PAGEIND,51605.08,0.70,-8.39,37.61,26.07,56415.0,-13740.0,150000.0,-32.53,32.0,X-MC,10.70,55.0,-0.24,1.08,0.70,X40,ATH,APPARELS


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,QUESS,424.00,-1.06,-30.86,106.60,42.84,47911.0,-20061.0,44945.0,-54.82,35.0,X-SC,37.32,83.0,-0.42,0.32,0.00,XY24,NTT,MISC
59,RELAXO,1176.00,0.73,-47.02,187.46,52.31,144177.0,-68249.0,76911.0,-43.57,39.0,X-SC,5.33,91.0,-0.47,0.55,2.95,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,-0.63,-37.35,115.80,35.20,93351.0,-48056.0,80614.0,5.70,25.0,X-SC,14.38,92.0,-0.51,0.58,0.68,X40,NTT,FOOTWEAR
22,DIXON,18931.72,-1.33,-11.17,36.87,21.58,45899.0,-15651.0,124488.0,-55.24,26.0,X-MC,5.73,56.0,-0.34,0.90,10.21,X40N,ATH,IT
51,MEDANTA,1486.00,-1.93,-0.72,24.98,24.08,31483.0,-916.0,126034.0,-8.70,37.0,X-SC,5.96,89.0,-0.03,0.91,18.75,XY24,NTT,HEALTHCARE


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.97,1.41,-13.60,38.05,19.28,113739.0,-47036.0,298920.0,-24.60,64.0,X-LC,6.73,1.0,-0.41,2.15,10.51,X40,ATH,IT
41,INFY,2275.00,1.06,8.61,42.60,54.87,144763.0,26932.0,339820.0,-15.29,67.0,X-LC,2.98,2.0,0.19,2.45,17.60,X40,BTT,IT
75,TMPV,600.00,-0.15,-15.57,68.40,42.18,159629.0,-43034.0,233376.0,-24.30,32.0,X-LC,3.55,3.0,-0.27,1.68,1.09,XY24,NTT,AUTO
81,VBL,671.64,1.14,-2.44,39.07,35.67,120383.0,-7720.0,308122.0,-14.05,61.0,X-LC,5.29,4.0,-0.06,2.22,11.07,X40N,ATH,FMCG
43,ITC,452.00,0.36,-1.79,12.45,10.44,24471.0,-3584.0,196554.0,-41.15,43.0,X-LC,1.58,5.0,-0.15,1.41,3.67,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.00,-0.03,-15.10,114.00,81.67,90791.0,-14169.0,79641.0,7161.11,44.0,L-SC,16.41,271.0,-0.16,0.57,54.23,XR,NTT,CERAMICS
50,MASFIN,398.61,-2.92,-4.87,28.29,22.05,26369.0,-4770.0,93210.0,-17.80,50.0,H-SC,6.85,164.0,-0.18,0.67,35.49,XR,ATH,FINANCE
13,BSOFT,831.70,2.51,-21.43,91.92,50.79,101575.0,-30149.0,110504.0,0.89,74.0,H-SC,5.56,151.0,-0.30,0.80,28.66,XR,ATH,IT
69,SURYODAY,216.00,-0.96,-20.51,54.77,23.03,77966.0,-36720.0,142351.0,55.70,46.0,H-SC,10.07,167.0,-0.47,1.02,41.24,XR,NTT,BANKS
39,INDIGOPNTS,1408.00,1.13,-7.65,8.30,0.02,13381.0,-13347.0,161212.0,146.09,68.0,M-SC,15.57,234.0,-1.00,1.16,40.23,OX40N,NTT,PAINTS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDIGOPNTS,1408.00,1.13,-7.65,8.30,0.02,13381.0,-13347.0,161212.0,146.09,68.0,M-SC,15.57,234.0,-1.00,1.16,40.23,OX40N,NTT,PAINTS
5,ASIANPAINT,3460.25,0.18,3.42,16.94,20.95,44109.0,8615.0,260383.0,2.32,79.0,X-LC,17.94,31.0,0.20,1.87,40.28,X40,ATH,PAINTS
13,BSOFT,831.70,2.51,-21.43,91.92,50.79,101575.0,-30149.0,110504.0,0.89,74.0,H-SC,5.56,151.0,-0.30,0.80,28.66,XR,ATH,IT
40,INDUSINDBK,1800.00,-0.40,-37.48,112.54,32.89,54326.0,-28934.0,48273.0,-696.43,59.0,L-MC,6.00,259.0,-0.53,0.35,32.94,XR,NTT,BANKS
41,INFY,2275.00,1.06,8.61,42.60,54.87,144763.0,26932.0,339820.0,-15.29,67.0,X-LC,2.98,2.0,0.19,2.45,17.60,X40,BTT,IT


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.39
1,25,45.25
2,50,76.18


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    46.21
MC    29.48
LC    24.28
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     27.96
X40      21.88
X40N     12.17
XR        9.95
AR        9.26
XY25      9.21
OX40N     7.71
SR        1.02
MH        0.81
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    24.39
X-MC    22.89
X-LC    20.79
M-SC    12.05
X-SC     8.37
H-MC     4.84
M-MC     1.40
L-SC     1.40
M-LC     1.29
H-LC     1.20
L-LC     1.00
L-MC     0.35
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,17.07,-6.13,40.45
IT,12.52,-15.91,78.58
FINANCE,9.46,-17.80,67.58
MISC,7.60,-22.82,73.14
ELECTRICAL,6.15,-9.59,49.73
PAINTS,6.15,-8.11,24.17
INSURANCE,4.44,-2.42,38.52
PHARMA,3.98,-0.88,33.17
AUTO,3.41,-21.39,73.43


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3150858.0,21
AR,1282919.0,10
XR,1280032.0,13
X40,996556.0,14
X40N,758059.0,9
OX40N,719095.0,10
XY25,361282.0,6
SR,277908.0,2
MH,72221.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3590308.0,25
M-SC,1381563.0,15
X-MC,1301873.0,16
X-LC,873515.0,11
X-SC,756612.0,8
H-MC,404750.0,3
L-SC,265373.0,3
M-LC,112939.0,1
H-LC,66474.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1258339.0      6
           AR         882436.0      5
M-SC       XY24       791210.0      6
H-SC       XR         766654.0      7
X-MC       X40        450345.0      7
           XY24       387412.0      3
X-LC       X40        367892.0      5
H-SC       OX40N      332750.0      4
M-SC       OX40N      299413.0      5
X-SC       X40N       290453.0      3
           XY24       287840.0      3
H-SC       SR         277908.0      2
X-MC       X40N       273477.0      4
X-LC       XY24       232048.0      2
H-MC       AR         210741.0      2
X-LC       X40N       194129.0      2
H-MC       XY24       194009.0      1
X-MC       XY25       190639.0      2
L-SC       XR         178441.0      2
X-SC       X40        178319.0      2
M-SC       XR         167672.0      2
           AR         123268.0      2
M-LC       XR         112939.0      1
L-SC       OX40N       86932.0      1
X-LC       XY25        79446.0      2
H-SC       MH          72221.0      1
H-LC       AR          66474.0      1
L-MC       XR          54326.0      1
M-MC       XY25        51127.0      1
L-LC       XY25        40070.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 33.0 seconds
